In [25]:
import mido
import sys
sys.path.append('../')

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import utils
from torchvision import datasets, transforms
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from copy import deepcopy
%matplotlib inline

In [26]:
def durationClass(duration):
    if (duration <= 0.25):
        return 0
    if (duration < 0.45):
        return 1
    if (duration < 0.6):
        return 2
    if (duration < 0.8):
        return 3
    return 4

In [27]:
def GetNums(mid):
    durations = dict.fromkeys([i for i in range(128)], 0)
    timer = 0.0
    lastAppear = []
    for i in range(128):
        lastAppear.append(0)
        musicToIndexes = []
    currentIndex = 0;
    for msg in mid.play(meta_messages=True):
        timer += msg.time
        if (msg.type == "note_on"):
            musicToIndexes.append(msg.note)
            durations[msg.note] = timer
            lastAppear[msg.note] = currentIndex
            currentIndex += 1
        if (msg.type == "note_off"):
            duration = timer - durations[msg.note]
            typeDuration = durationClass(duration)
            musicToIndexes[lastAppear[msg.note]] = msg.note + 128* typeDuration
    return musicToIndexes, mid.ticks_per_beat

In [28]:
def indexToNote(index):
    decoder = {}
    for i in range(128):
        for j in range(5):
            decoder[i + j*128] = [i, j]
    return decoder[index]

In [29]:
from mido import Message, MidiFile, MidiTrack
def toMidi(Indexes, ticks):
    #print(Indexes)
    moments = [[] for i in range(len(Indexes) + 4)]
    current = 0
    for note in Indexes:
        decoded = indexToNote(note)
        moments[current].append(decoded[0])
        #print(current, decoded[1], len(Indexes))
        moments[current + decoded[1]].append(-decoded[0])
        current += 1
    newMid = MidiFile()
    track = MidiTrack()
    newMid.tracks.append(track)
    tick_time = round(ticks* 0.1) * 2
    for moment in moments:
        for ivent in moment:
            if (ivent < 0):
                track.append(Message('note_off', note=-ivent, velocity=0, time=tick_time))
            else:
                track.append(Message('note_on', note=ivent, velocity=64, time=tick_time))
    return newMid

In [30]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

# device = device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#print(device)

In [31]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size)

In [36]:
teacher_forcing_ratio = 1


def train(input_tensor, target_tensor, decoder, decoder_optimizer, criterion, max_length=1000):
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)


    loss = 0


    decoder_input = torch.LongTensor([[SOS_token]])

    decoder_hidden = input_tensor

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            #print(decoder_input)
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di:di+1])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input
            loss += criterion(decoder_output, target_tensor[di:di+1])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    decoder_optimizer.step()

    return loss.item() / target_length

In [33]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [39]:
def trainIters(n_iters, musicIndexes, decoder, print_every=20, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    
    criterion = nn.NLLLoss()
 
        
    for iter in range(1, n_iters + 1):
        input_tensor = []
        for i in range(hidden_size):
            input_tensor.append(random.randint(1, 10))
        input_tensor = np.reshape(input_tensor, (1, 1, hidden_size))
        input_tensor = torch.Tensor(input_tensor.tolist())
        x = musicIndexes[iter%12]
        x = torch.LongTensor(x)
        #print(x)
        loss = train(input_tensor, x, decoder, decoder_optimizer, criterion)
        print_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

    

#         if iter % plot_every == 0:
#             plot_loss_avg = plot_loss_total / plot_every
#             plot_losses.append(plot_loss_avg)
#             plot_loss_total = 0

#     showPlot(plot_losses)


In [75]:
import os
hidden_size = 256
output_size = 1000
SOS_token = 0
EOS_token = 1
musicInd = []
tracks =  [file for file in os.listdir("/home/maria/Documents/музычка/simpledata")]
middle_tick = 0
for file in tracks:
    mid1 = mido.MidiFile("/home/maria/Documents/музычка/simpledata/" +file)
    musicToIndexes1, ticks = GetNums(mid1)
    musicInd.append(musicToIndexes1)
    middle_tick += ticks



NameError: name 'music' is not defined

In [76]:
middle_tick /= len(musicInd)

In [77]:
# elements = 0
# for elem in musicInd:
#     size = 0
#     for el in elem:
#         size+=1
#     print(size)
#     elements += 1
#     print(elem)
# print(elements)
decoder = DecoderRNN(hidden_size, output_size)
trainIters(1000, musicInd, decoder)

0m 1s (- 0m 58s) (20 2%) 6.8901
0m 2s (- 1m 0s) (40 4%) 6.2851
0m 3s (- 1m 0s) (60 6%) 6.4852
0m 4s (- 0m 57s) (80 8%) 5.5340
0m 6s (- 0m 57s) (100 10%) 5.8142
0m 7s (- 0m 56s) (120 12%) 5.8260
0m 8s (- 0m 54s) (140 14%) 4.8918
0m 10s (- 0m 54s) (160 16%) 5.4124
0m 11s (- 0m 54s) (180 18%) 5.3404
0m 13s (- 0m 54s) (200 20%) 4.9011
0m 15s (- 0m 55s) (220 22%) 4.8949
0m 17s (- 0m 55s) (240 24%) 5.3875
0m 19s (- 0m 54s) (260 26%) 4.6340
0m 21s (- 0m 54s) (280 28%) 4.5702
0m 22s (- 0m 53s) (300 30%) 5.0116
0m 24s (- 0m 52s) (320 32%) 4.6099
0m 26s (- 0m 51s) (340 34%) 4.7530
0m 28s (- 0m 50s) (360 36%) 5.4230
0m 30s (- 0m 49s) (380 38%) 4.8984
0m 31s (- 0m 47s) (400 40%) 4.6678
0m 33s (- 0m 46s) (420 42%) 5.5885
0m 35s (- 0m 44s) (440 44%) 4.9383
0m 37s (- 0m 43s) (460 46%) 4.7654
0m 39s (- 0m 42s) (480 48%) 5.0973
0m 40s (- 0m 40s) (500 50%) 5.3735
0m 42s (- 0m 39s) (520 52%) 4.9131
0m 44s (- 0m 37s) (540 54%) 7.2161
0m 45s (- 0m 35s) (560 56%) 6.0669
0m 47s (- 0m 34s) (580 57%) 5.5519
0m

In [9]:
def evaluate(input_tensor, decoder, prob, max_length=100):
    with torch.no_grad():
        
        input_tensor = torch.FloatTensor(input_tensor)
        decoder_input = torch.tensor([[SOS_token]])  # SOS

        decoder_hidden = input_tensor

        decoded_words = []

        
        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(prob)
            item = random.randint(0, prob - 1)
            
            topv = topv[0][item]
            topi = topi[0][item]
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(topi.item())

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [42]:
def generate_sample():
    begin = []
    note = random.randint(1, 10)
    for i in range(256):
        begin.append(note)
#     for i in range(256):
#         begin.append(random.randint(200, 240))
    begin = np.reshape(begin, (1,1, 256))
    print("begin_predict")
    generated = evaluate(begin, decoder, 20)
    mid = toMidi(generated, middle_tick)
    mid.save('generated1.mid')
#generate_sample()

In [96]:
torch.save(decoder, "/home/maria/Documents/Sonia/decoder")

/home/maria/anaconda3/lib/python3.5/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type DecoderRNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [17]:
hidden_size = 256
output_size = 1000
decoder = DecoderRNN(hidden_size, output_size)
decoder = torch.load("/home/maria/Documents/Sonia/decoder")

In [22]:
hidden_size = 256
output_size = 1000
SOS_token = 0
EOS_token = 1
middle_tick = 48*16
generate_sample()

begin_predict


In [35]:
import os
hidden_size = 256
output_size = 1000
SOS_token = 0
EOS_token = 1
musicInd = []
tracks =  [file for file in os.listdir("/home/maria/Downloads/chaikovskiy/tschai")]
middle_tick = 0
for file in tracks:
    mid1 = mido.MidiFile("/home/maria/Downloads/chaikovskiy/tschai/" +file)
    musicToIndexes1, ticks = GetNums(mid1)
    musicInd.append(musicToIndexes1)
    middle_tick += ticks

middle_tick /= len(musicInd)
# elements = 0
# for elem in musicInd:
#     size = 0
#     for el in elem:
#         size+=1
#     print(size)
#     elements += 1
#     print(elem)
# print(elements)


In [38]:
print(len(musicInd))

12


In [40]:
decoder = DecoderRNN(hidden_size, output_size)
trainIters(1000, musicInd, decoder)

1m 29s (- 72m 51s) (20 2%) 4.4111
2m 58s (- 71m 32s) (40 4%) 4.1384
4m 32s (- 71m 7s) (60 6%) 3.9887
6m 9s (- 70m 47s) (80 8%) 3.7589
7m 38s (- 68m 46s) (100 10%) 3.5785
9m 11s (- 67m 23s) (120 12%) 3.4724
10m 47s (- 66m 19s) (140 14%) 3.5061
12m 18s (- 64m 39s) (160 16%) 3.4317
13m 54s (- 63m 20s) (180 18%) 3.4218
15m 31s (- 62m 6s) (200 20%) 3.4535
17m 3s (- 60m 28s) (220 22%) 3.7220
18m 37s (- 58m 57s) (240 24%) 3.9594
20m 14s (- 57m 36s) (260 26%) 4.3618
21m 43s (- 55m 52s) (280 28%) 4.2166
23m 17s (- 54m 20s) (300 30%) 4.0377
24m 54s (- 52m 56s) (320 32%) 3.9607
26m 23s (- 51m 14s) (340 34%) 3.9729
27m 56s (- 49m 41s) (360 36%) 3.6346
29m 33s (- 48m 13s) (380 38%) 3.6113
31m 3s (- 46m 34s) (400 40%) 3.7435
32m 38s (- 45m 4s) (420 42%) 3.9586
34m 16s (- 43m 37s) (440 44%) 3.9106
35m 46s (- 41m 59s) (460 46%) 3.7438
37m 18s (- 40m 24s) (480 48%) 3.7087
38m 55s (- 38m 55s) (500 50%) 3.5003
40m 25s (- 37m 18s) (520 52%) 3.4753
41m 58s (- 35m 45s) (540 54%) 3.5220
43m 34s (- 34m 14s) (

In [43]:
hidden_size = 256
output_size = 1000
SOS_token = 0
EOS_token = 1
generate_sample()

begin_predict


In [45]:
print(middle_tick)

480.0


In [44]:
torch.save(decoder, "/home/maria/Documents/Sonia/decoder_tchaik")

/home/maria/anaconda3/lib/python3.5/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type DecoderRNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [47]:
decoder = torch.load("/home/maria/Documents/Sonia/decoder_tchaik")

In [48]:
print(decoder)

DecoderRNN(
  (embedding): Embedding(1000, 256)
  (gru): GRU(256, 256)
  (out): Linear(in_features=256, out_features=1000, bias=True)
  (softmax): LogSoftmax()
)
